In [16]:
import uuid
import random

from tqdm import tqdm
from threading import Thread
from threading import Thread, Lock

from cassandra.cluster import Cluster, BatchStatement
from cassandra.policies import RetryPolicy, ExponentialReconnectionPolicy

from utils.query_utils import *
from cassandra import ConsistencyLevel
from cassandra.cluster import ExecutionProfile, EXEC_PROFILE_DEFAULT

from cassandra.cluster import Cluster
from cassandra import ConsistencyLevel
from cassandra.cluster import Cluster, ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra.policies import WhiteListRoundRobinPolicy, DowngradingConsistencyRetryPolicy
from cassandra.query import tuple_factory
import time

In [17]:
profile = ExecutionProfile(
    retry_policy = DowngradingConsistencyRetryPolicy,
    consistency_level = ConsistencyLevel.ALL,
    serial_consistency_level = ConsistencyLevel.LOCAL_SERIAL,
    request_timeout = 200,
)

In [18]:
CLUSTER_IDS = ['172.21.0.2']
KEYSPACE = 'library_keyspace'

cluster = Cluster(CLUSTER_IDS, execution_profiles={EXEC_PROFILE_DEFAULT: profile})
session = cluster.connect()
session.set_keyspace(KEYSPACE)

In [19]:
res = get_all_reservations(session)
for r in res:
    print(r.reservation_id, r.user_id, r.user_name, r.book_name)
    reservation_id = r.reservation_id
    
reservation_id

b4ad11fb-066b-436d-8e08-44fd7fb6e611 f581cef3-7a10-4ee7-968c-bc90f96d8b9b User 101 Book 2001


UUID('b4ad11fb-066b-436d-8e08-44fd7fb6e611')

In [20]:
res = list(get_all_books(session))
book_id, book_name = None, None
print(len(res))
for r in res:
    print(r.book_id, r.book_name, r.is_reserved)
    # if r.is_reserved:
    #     print(r.book_id, r.book_name, r.is_reserved)
    if r.book_name == "Book 1":
        book_id = r.book_id
        book_name = "Book 1"

book_id, book_name

2001
8cd8e91a-515f-4797-aad4-a05762c3e844 Book 231 False
6ebc161a-43d9-4a8f-be89-b2924d67e062 Book 1832 False
98e36945-f532-482b-b856-e0bcd962691a Book 1896 False
3e520ce0-3c95-45fe-adf7-09bcab5aa51d Book 123 False
00b94874-7985-4a93-affa-cd6ca918304b Book 1378 False
05f552b2-7828-4017-8eb0-28707b03019a Book 594 False
b03ef0e7-c362-463d-b7f0-4b8408610de5 Book 453 False
4781d699-a1c1-4f7c-a97b-230d151397dd Book 143 False
1167e13a-bc66-4583-b788-44236bbb5032 Book 1886 False
af578c85-2841-4b28-a1df-f0f24279a6b9 Book 552 False
49a24083-c7df-4272-9a09-88e5991acfac Book 897 False
b1c415ca-b30d-4c17-9033-0b533b48bad7 Book 1610 False
e1d3904d-886f-4b8b-92fc-b3f9a6c9ed09 Book 1424 False
5c86f57b-ba50-4728-a030-640ee1545859 Book 1618 False
ead9c9ed-910d-4af9-b256-753572edf2b7 Book 1190 False
2a815977-cdd0-4de0-bec8-4b2c1020a4bd Book 180 False
7c599bf6-eaad-4720-aaf6-eb516ba027c6 Book 1889 False
b883d3d9-0647-41ee-b2e4-7826111b2ca1 Book 1182 False
25baf34e-e8dd-41c3-9ab1-e45625bfc47b Book 1284 Fa

(UUID('e1e47dc3-25ca-4d8b-bbde-a6c5cf932da2'), 'Book 1')

In [21]:
res = list(get_all_users(session))
for r in res:
    if r.user_name == "User 1":
        user_name = "User 1"
        user_id = r.user_id
    # if r.reservation_ids_list is not None:
    #     print(r.user_id, r.user_name, r.reservation_ids_list)
    print(r.user_id, r.user_name, r.reservation_ids_list)

user_name, user_id

0a15c306-6bf1-48ff-9e08-3fb1422d6a4a User 10 None
d85ce4ce-a7a7-4ba8-835b-959391c15a51 User 25 None
063b0019-ad39-4179-906d-8a382cc75aa0 User 32 None
25b9463a-5efb-4561-ad2d-c9346fec1d0c User 64 None
81c6de77-a44f-4418-82d1-403cb27a85e5 User 63 None
f12fd1cd-d935-42bd-b1b1-02739c6084d4 User 14 None
f5055545-754d-44e4-89dc-b5cef5209abc User 39 None
e89a23d9-0dd4-44f8-8065-a4be72085eec User 85 None
855615db-8040-4cec-ae89-119e30768469 User 66 None
8f8a2fc7-d3eb-4d08-8992-ebce991cdd30 User 97 None
06924790-3ae5-49e6-abbc-a8fb08482dd2 User 93 None
c94727e8-5885-4396-8d5e-345828f1ab46 User 81 None
baecfd0b-4c2f-403c-8233-74f53b0dc912 User 65 None
c736fe0c-5893-49c8-b736-4814925b8a53 User 22 None
fe88d228-2ccf-461f-873a-b8781b0e0471 User 78 None
c592e511-c46a-4ee6-a056-dfb9bce22dd1 User 9 None
5911bb3d-c743-4eed-bce7-79dfa52377bf User 55 None
c635d066-12f3-4988-97c1-ab85d0f7f34f User 21 None
346c06f1-7454-4c87-85a3-f3ef4a66fcda User 57 None
59fe45ed-7b98-428a-8f02-0e72900e3df6 User 67 None
2

('User 1', UUID('b03f87bb-e73a-4d11-9119-7a01372994b6'))

In [22]:
def test_reserve_cancel_debug(session, reservation_id, user_id, user_name, book_id, book_name, repeat_num=10_000):
    # Creates and cancels a reservation 10_000 times

    # for _ in range(repeat_num):
    for _ in range(3):
        add_reservation(session, reservation_id, user_id, user_name, book_name, book_id)

        print("Created")
        res = get_all_reservations(session)
        for r in res:
            if r.reservation_id == reservation_id:
                print(r.reservation_id, r.user_id, r.user_name, r.book_name)

        cancel_reservation(session, reservation_id)
        print("Cancelled")
        res = get_all_reservations(session)
        for r in res:
            if r.reservation_id == reservation_id:
                print(r.reservation_id, r.user_id, r.user_name, r.book_name)


def test_update_reservations_debug(session, num_reservations=1000):
    reservations = list(get_all_reservations(session))
    books = get_all_books(session)
    available_books = []
    for book in books:
        if not book.is_reserved:
            available_books.append(book)

    assert len(reservations) >= num_reservations, f"There should be at least a {num_reservations} existing reservations"
    assert len(available_books) >= num_reservations, f"There should be at least a {num_reservations} available books"

    for i in tqdm(range(num_reservations)):
        reservation = reservations[i]
        new_book = available_books[i]

        assert reservation.book_id != new_book.book_id
        assert reservation.book_name != new_book.book_name

        update_reservation_book(session, reservation.reservation_id, new_book.book_id)
        
        updated_reservation = get_reservation_by_id(session, reservation.reservation_id)
        
        assert updated_reservation.book_id == new_book.book_id
        assert updated_reservation.book_name == new_book.book_name

def test_make_random_requests_two_users_debug(user1, user2, num_repeats=10_000):
    # Simulates two users making 10_000 total random requests

    def make_random_requests(user_id, user_name, num_actions):
        # Simulates a user making random requests num_actions number of times

        user_cluster = Cluster(CLUSTER_IDS)
        user_session = user_cluster.connect(KEYSPACE)

        possible_requests = ["update_reservation_book", "update_reservation_user", "cancel_reservation", "make_reservation"]

        while num_actions > 0:

            selected_action = random.choice(possible_requests)

            # Update DB state information
            reservations = list(get_all_reservations(user_session))

            # If no reservations left, make a reservation
            if len(reservations) == 0:
                selected_action = "make_reservation"

            books = list(get_all_books(user_session))
            users = list(get_all_users(user_session))

            if selected_action == "update_reservation_book":
                reservation = random.choice(reservations)
                book = random.choice(books)

                if user_name == "User 1":
                    print(f"User1 Num actions left: {num_actions}")
                    
                else:
                    print(f"User2 Num actions left: {num_actions}")

                print("   Action:", selected_action)
                print("   Reservation:", reservation)
                print("   Book:", book)
                print("============================")
                    
                # Does not assume book is available
                update_reservation_book(user_session, reservation.reservation_id, book.book_id)

            elif selected_action == "update_reservation_user":
                reservation = random.choice(reservations)
                user = random.choice(users)
                
                if user_name == "User 1":
                    print(f"User1 Num actions left: {num_actions}")

                else:
                    print(f"User2 Num actions left: {num_actions}")

                print("   Action:", selected_action)
                print("   Reservation:", reservation)
                print("   User:", user)
                print("============================")

                # Does not assume user does not possess reservation already
                update_reservation_user(user_session, reservation.reservation_id, user.user_id)

            elif selected_action == "cancel_reservation":
                reservation = random.choice(reservations)
                
                if user_name == "User 1":
                    print(f"User1 Num actions left: {num_actions}")

                else:
                    print(f"User2 Num actions left: {num_actions}")
                
                print("   Action:", selected_action)
                print("   Reservation:", reservation)
                print("============================")

                cancel_reservation(user_session, reservation.reservation_id)

            elif selected_action == "make_reservation":
                available_books = [book for book in books if not book.is_reserved]

                if user_name == "User 1":
                    print(f"User1 Num actions left: {num_actions}")

                else:
                    print(f"User2 Num actions left: {num_actions}")

                print("   Action:", selected_action)
                print("   Available books:", len(available_books))

                # There are books available
                if len(available_books) > 0:
                    selected_random_book = random.choice(available_books)
                    print("   Selected book:", selected_random_book)

                    try:
                        add_reservation(user_session, uuid.uuid4(), user_id, user_name, selected_random_book.book_name, selected_random_book.book_id)
                    except:
                        pass
                    
                # No available books, dont use action
                else:
                    num_actions += 1

                print("============================")

            num_actions -= 1

        user_session.shutdown()

    thread1 = Thread(target=make_random_requests, args=[user1.user_id, user1.user_name, num_repeats//2])
    thread2 = Thread(target=make_random_requests, args=[user2.user_id, user2.user_name, num_repeats//2])

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()

In [23]:
def test_same_request(session, reservation_id, new_book_id, repeat_num=10_000):
    # Runs an update of the book in a reservation 10_000 times

    for _ in tqdm(range(repeat_num)):
        update_reservation_book(session, reservation_id, new_book_id)


def test_reserve_cancel(session, reservation_id, user_id, user_name, book_id, book_name, repeat_num=10_000):
    # Creates and cancels a reservation 10_000 times

    for _ in tqdm(range(repeat_num)):
        add_reservation(session, reservation_id, user_id, user_name, book_name, book_id)
        cancel_reservation(session, reservation_id)

    add_reservation(session, reservation_id, user_id, user_name, book_name, book_id)


def test_update_reservations(session, num_reservations=1000):
    # Updates a 1000 reservations

    reservations = list(get_all_reservations(session))
    books = get_all_books(session)
    available_books = []
    for book in books:
        if not book.is_reserved:
            available_books.append(book)

    assert len(reservations) >= num_reservations, f"There should be at least a {num_reservations} existing reservations"
    assert len(available_books) >= num_reservations, f"There should be at least a {num_reservations} available books"

    for i in tqdm(range(num_reservations)):
        reservation = reservations[i]
        new_book = available_books[i]

        update_reservation_book(session, reservation.reservation_id, new_book.book_id)

lock = Lock()

# TODO: currently the same book can be reserved by two users.
# Add random wait time before checking availability/locks/idk
def test_make_all_reservations_two_users(user1, user2):
    # Simulates two users reserving all possible books

    def make_all_reservations(user_id, user_name):
        # Simulates user making all possible reservations one by one

        user_cluster = Cluster(CLUSTER_IDS)
        user_session = user_cluster.connect(KEYSPACE)

        # Update available book state
        books = get_all_books(user_session)
        available_books = [book for book in books if not book.is_reserved]
    
        while len(available_books) > 0:
            selected_random_book = random.choice(available_books)
            
            try:
                with lock:
                    add_reservation(user_session, uuid.uuid4(), user_id, user_name, selected_random_book.book_name, selected_random_book.book_id)
                    time.sleep(random.uniform(0.01, 0.1))
            except:
                pass
            
            with lock:
                books = get_all_books(user_session)
                available_books = [book for book in books if not book.is_reserved]

        user_session.shutdown()

    thread1 = Thread(target=make_all_reservations, args=[user1.user_id, user1.user_name])
    thread2 = Thread(target=make_all_reservations, args=[user2.user_id, user2.user_name])

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()

def test_make_random_requests_two_users(user1, user2, num_repeats=10_000):
    # Simulates two users making 10_000 total random requests

    def make_random_requests(user_id, user_name, num_actions):
        # Simulates a user making random requests num_actions number of times

        user_cluster = Cluster(CLUSTER_IDS)
        user_session = user_cluster.connect(KEYSPACE)

        possible_requests = ["update_reservation_book", "update_reservation_user", "cancel_reservation", "make_reservation"]

        while num_actions > 0:

            selected_action = random.choice(possible_requests)

            # Update DB state information
            reservations = list(get_all_reservations(user_session))

            # If no reservations left, make a reservation
            if len(reservations) == 0:
                selected_action = "make_reservation"

            books = list(get_all_books(user_session))
            users = list(get_all_users(user_session))
            
            if selected_action == "update_reservation_book":
                reservation = random.choice(reservations)
                book = random.choice(books)

                # Does not assume book is available
                update_reservation_book(user_session, reservation.reservation_id, book.book_id)

            elif selected_action == "update_reservation_user":
                reservation = random.choice(reservations)
                user = random.choice(users)

                # Does not assume user does not possess reservation already
                update_reservation_user(user_session, reservation.reservation_id, user.user_id)

            elif selected_action == "cancel_reservation":
                reservation = random.choice(reservations)
                cancel_reservation(user_session, reservation.reservation_id)

            elif selected_action == "make_reservation":
                available_books = [book for book in books if not book.is_reserved]

                # There are books available
                if len(available_books) > 0:
                    selected_random_book = random.choice(available_books)

                    try:
                        add_reservation(user_session, uuid.uuid4(), user_id, user_name, selected_random_book.book_name, selected_random_book.book_id)
                    except:
                        pass
                    
                # No available books, dont use action
                else:
                    num_actions += 1

            num_actions -= 1

        user_session.shutdown()

    thread1 = Thread(target=make_random_requests, args=[user1.user_id, user1.user_name, num_repeats//2])
    thread2 = Thread(target=make_random_requests, args=[user2.user_id, user2.user_name, num_repeats//2])

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()

In [24]:
users = get_all_users(session)
user1, user2 = None, None
for user in users:
    if user.user_name == "User 1":
        user1 = user

    if user.user_name == "User 2":
        user2 = user

user1, user2

(Row(user_id=UUID('b03f87bb-e73a-4d11-9119-7a01372994b6'), reservation_ids_list=None, user_name='User 1'),
 Row(user_id=UUID('ed3e6f94-f114-422b-af41-223d8cdc5379'), reservation_ids_list=None, user_name='User 2'))

In [25]:
test_make_random_requests_two_users(user1, user2, 10_000)

In [26]:
test_make_all_reservations_two_users(user1, user2)

In [27]:
all_reservations = get_all_reservations(session)

user_1_reservations = 0
user_2_reservations = 0
for reservation in all_reservations:
    if reservation.user_id == user1.user_id:
        user_1_reservations += 1

    if reservation.user_id == user2.user_id:
        user_2_reservations += 1

print(f"User 1 reservation count: {user_1_reservations}")
print(f"User 2 reservation count: {user_2_reservations}")

User 1 reservation count: 999
User 2 reservation count: 1001


In [28]:
res = list(get_all_reservations(session))
user_1_reservations = [r for r in res if r.user_id == user1.user_id]
user_2_reservations = [r for r in res if r.user_id == user2.user_id]

user_1 = get_user(session, user1.user_id)
user_2 = get_user(session, user2.user_id)

for r in user_1_reservations:
    if r in user_2_reservations:
        print("Error user1 reservation in user2 reservations list")

    if r.reservation_id not in user_1.reservation_ids_list:
        print("Error user1 reservation not in list")

for r in user_2_reservations:
    if r in user_1_reservations:
        print("Error user2 reservation in user1 reservations list")

    if r.reservation_id not in user_2.reservation_ids_list:
        print("Error user2 reservation not in list")

In [29]:
book_ids_1 = set([r.book_id for r in user_1_reservations])
book_ids_2 = set([r.book_id for r in user_2_reservations])

assert len(book_ids_1 & book_ids_2) == 0, "No book should be reserved by both clients at the same time"

In [30]:
book_ids_1 & book_ids_2

set()

In [31]:
book = list(book_ids_1 & book_ids_2)[0]
for res in get_all_reservations(session):
    if res.book_id == book:
        print(res.reservation_id, res.user_name, res.book_name, res.book_id)

IndexError: list index out of range

In [ ]:
books = get_all_books(session)
available_books = [book for book in books if not book.is_reserved]
available_books

[]

In [ ]:
test_update_reservations(session)

AssertionError: There should be at least a 1000 available books

In [ ]:
test_reserve_cancel(session, reservation_id, user_id, user_name, book_id, book_name)

res = get_all_reservations(session)
for r in res:
    print(r.reservation_id, r.user_id, r.user_name, r.book_name)
    

100%|██████████| 10000/10000 [03:33<00:00, 46.90it/s]


44c4f1be-d266-437b-9dcf-bbfe45eec299 a26c83c2-3a1b-4f26-b18c-a0a8385bc492 User 1 Book 1001


In [ ]:
test_same_request(session, reservation_id, book_id)

res = get_all_reservations(session)
for r in res:
    print(r.reservation_id, r.user_id, r.user_name, r.book_name)

100%|██████████| 10000/10000 [00:44<00:00, 226.46it/s]

44c4f1be-d266-437b-9dcf-bbfe45eec299 a26c83c2-3a1b-4f26-b18c-a0a8385bc492 User 1 Book 1


In [ ]:
session.shutdown()
cluster.shutdown()